In [ ]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

#### - Get every serie paths and generate JSON file 

In [ ]:
series_paths = get_series_path('/media/deeplearning/LACIE SHARE/good')
export_folder = '/media/deeplearning/LACIE SHARE/good/JSON'

In [ ]:
index_problem = []
for serie_path in series_paths:
    print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

In [ ]:
json_path = export_folder
list_dir_json = os.listdir(json_path)
list_json = []
for file_ in list_dir_json : 
    list_json.append([os.path.join(json_path, file_)])

#### - Get informations from every json in list 

In [ ]:
for liste in list_json : 
    #print(list_json.index(liste))
    with open(liste[0]) as json_file : 
        reader = json.load(json_file)

        patient_id = reader["patient"]["PatientID"]
        #patient_id = reader['path'].split('/')[6]
        study_uid = reader["study"]['StudyInstanceUID']
        modal = reader['series']['Modality']

        path = reader['path']

        type_ = reader['study']["AccessionNumber"]
        #print(type_)
        #date = reader['study']['StudyDate']
        #name = '_'.join(reader['patient']['PatientName'].split('^')).lower()
        liste.append(path)
        liste.append(modal)
        liste.append(type_)
        liste.append(patient_id)
        #liste.append(name)
        liste.append(study_uid)
        #liste.append(date)

In [ ]:
list_json[0]

#### - Keep only pet0/pet screening 

In [ ]:
not_pet_screening = []
for data in list_json : 
    #print(data[3])
    if data[3] != 'PET0' : 
        not_pet_screening.append(data)
print(len(not_pet_screening))
for serie in not_pet_screening : 
    list_json.remove(serie)

In [ ]:
print("Nbr de serie pet0/screening :", len(list_json))

#### - Reunite CT/PT series with same study uid 

In [ ]:
#get every study_uid 
liste = []
for data in list_json: 
    liste.append(data[-2])
#print(liste)
study_uid = []
study_uid.append(liste[0])
for uid in liste : 
    if uid not in study_uid : 
       study_uid.append(uid)

In [ ]:
print('Nombre de study :', len(study_uid))

In [ ]:
#find serie for each study uid 
data = []
for uid in study_uid : 
    #print(uid)
    sub = []
    for serie in list_json : 
        if uid in serie: 
            #print(serie)
            sub.append(serie)
    data.append(sub)


In [ ]:
dataset = []
for serie in data : 
    #print(serie)
    if len(serie) != 1 : 
        serie[0].append(serie[1][0])
        serie[0].append(serie[1][1])
        serie[0].append(serie[1][2])

    dataset.append(serie[0])

In [ ]:
dataset[0]

#### Clean dataset : if there is no json, double study_uid, if not CT AND PT

In [ ]:
#no json 
i = 0
no_json_liste = []
for data in dataset : 
    if len(data) != 9 : 
        i += 1 
        no_json_liste.append(data)

if i != 0 : 
    print("number of study with no json : {}".format(i))

    for no_json_file in no_json_liste : 
        no_json_liste.remove(no_json_file)

    print("dataset cleaned ")

In [ ]:
#double study_uid 
double = []
double.append(dataset[0])
for data in dataset : 
    if data not in double : 
        double.append(data)

if len(double) == len(dataset) : 
    print('No double study uid, dataset cleaned')

In [ ]:
#check Modality 
index = []
for data in dataset : 
    modal = []
    modal.append(data[2])
    modal.append(data[-1])
    #print(modal)
    if modal[0] == modal[1] : 
        index.append(dataset.index(data))
#print(index)
#if index is not empty : 
if len(index) != 0 :
    liste_to_remove = []
    for ind in index : 
        liste_to_remove.append(dataset[ind])

    for r in liste_to_remove : 
        dataset.remove(r)

print("dataset cleaned")

In [ ]:
print("Nombre de study après clean :", len(dataset))

#### - Sorted  by date studies from 1 patient if necessary 

In [ ]:
liste_patient_id = []
for data in dataset : 
    if data[3] not in liste_patient_id :
        liste_patient_id.append(data[3])

print("Nombre de patients : ", len(liste_patient_id))

In [ ]:
liste_pet_to_sort = []
for patient_id in liste_patient_id : 
    liste = []
    for data in dataset : 
        if patient_id == data[3] : 
            liste.append(data)

    liste_pet_to_sort.append(liste)

In [ ]:
len(liste_pet_to_sort)

In [ ]:
from datetime import datetime 
import numpy as np
for patient in liste_pet_to_sort : 
    if len(patient) != 1 : 
        #liste_date = []
        for i in range(len(patient)):
            #liste_date.append(datetime.strptime(patient[i][5], "%Y%m%d"))
            patient[i][5] = datetime.strptime(patient[i][5], "%Y%m%d")
        #print(patient)
        patient.sort(key=lambda patient:patient[5])
        for i in range(len(patient)):
            patient[i][5] = datetime.strftime(patient[i][5], "%Y%m%d")
 

In [ ]:
liste_pet_to_sort[0]

In [ ]:
double_date = []
for patient in liste_pet_to_sort : 
    date_liste = []
    for i in range(len(patient)):
        date_liste.append(patient[i][5])

    if len(date_liste) != len(set(date_liste)) : 
        double_date.append(patient)


In [ ]:
len(double_date)

In [ ]:
write_json_file('/media/deeplearning/VERBATIM HD/LNH073B/LNH073B_Validated_DICOM', 'dataset_with_double_examen_v2', double_date)

In [ ]:
for pat in double_date :
    liste_pet_to_sort.remove(pat)

In [ ]:
#rewrite path with pet0
pet = ['pet0', 'pet2', 'pet4', 'fin']
for patient in liste_pet_to_sort : 
    for i in range(len(patient)) : 
        patient[i].append(pet[i])

In [ ]:
write_json_file('/media/deeplearning/VERBATIM HD/LNH073B/LNH073B_Validated_DICOM', 'dataset_validated_per_date_and_type_v2', liste_pet_to_sort)

#### - Prepare json file to generate nifti in "nifti_builder_from_json.ipynb" script

In [ ]:
study_results = []
for liste in dataset: 
    #for liste in patient : 
    subliste = []
        #print(liste)
    subliste.append(liste[1]) #path_1
    subliste.append(liste[2]) #modal 1
    subliste.append(liste[-2]) #path_2
    subliste.append(liste[-1]) #modal_2
    subliste.append(liste[5]) #study_uid
    subliste.append(liste[3]) #type
    subliste.append(liste[4]) #patient id
        #subliste.append(liste[5]) #patient name

    study_results.append(subliste)

In [ ]:
study_results[0]

In [ ]:
#save results as json 
directory = '/media/deeplearning/LACIE SHARE/good'
filename = 'ahl_dataset'
write_json_file(directory, filename, study_results)